In [82]:
import pandas as pd
import numpy as np
import os
import scipy.optimize as opt
import multiprocessing as mp;

from lib.noglobal import noglobal




In [2]:
@noglobal()
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

## Apply WLS on one collection and one measurement

In [3]:
root = "/work/data/input/google-smartphone-decimeter-challenge/"

collection_name="2020-05-29-US-MTV-1"
phone = 'Pixel4'

file_path = f"train/{collection_name}/"

measurement_epoch_time = 1274827487438

# baseline we'll compare our solution against
df_baseline = pd.read_csv(f"{root}/baseline_locations_train.csv")

# ground truth to compute methods performance
df_groundtruth = pd.read_csv(f"{root}/{file_path}/{phone}/ground_truth.csv")

# Train df here only contains one collection and one measurement
df_train = pd.read_csv(f"{root}/{file_path}/{phone}/{phone}_derived.csv")
df_train = df_train[df_train['millisSinceGpsEpoch'] == measurement_epoch_time] 

In [4]:
%%time
light_speed = 299_792_458

df_train['correctedPrM'] = df_train["rawPrM"] +  df_train["satClkBiasM"]  - df_train["isrbM"] - df_train["ionoDelayM"] - df_train["tropoDelayM"]
df_train['transmissionTimeSeconds'] = df_train['correctedPrM'] / light_speed

CPU times: user 18.3 ms, sys: 447 µs, total: 18.8 ms
Wall time: 16.6 ms


In [5]:
omega_e = 7.2921151467e-5
df_train["xSatPosMRotated"] = np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
df_train["ySatPosMRotated"] = - np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
df_train['zSatPosMRotated'] = df_train['zSatPosM']

In [6]:
df_train['uncertaintyWeight'] = 1 / df_train['rawPrUncM']

In [114]:
%%time
from tqdm.notebook import tqdm


@noglobal()
def distance(sat_pos, x):
    sat_pos_diff = sat_pos.copy(deep=True)
                
    sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
    sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
    sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]
    
    return sat_pos_diff["uncertaintyWeight"]*(np.sqrt((sat_pos_diff["xSatPosMRotated"]**2 + sat_pos_diff["ySatPosMRotated"]**2 + sat_pos_diff["zSatPosMRotated"]**2))+ x[3] - sat_pos_diff["correctedPrM"])

@noglobal()
def distance_fixed_satpos(x,train_df):    
    return distance(train_df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

@noglobal()
def predict_by_satelite_per_time(list_):
    key = list_[0]
    each_df = list_[1]
    
    x0= [0,0,0,0]        
    opt_res = opt.least_squares(lambda x: distance_fixed_satpos(x,each_df), x0)
    
    opt_res_pos = opt_res["x"]
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    return [key, wls_estimated_pos[0],wls_estimated_pos[1],wls_estimated_pos[2]]
    
@noglobal()
def multi(function_name,exe_list):
    p = mp.Pool(mp.cpu_count()-1);
    retult = p.map(function_name,exe_list);
    p.close();
    return retult    

@noglobal()
def predict_by_gnss_data(baseline_train_df,collection,phone):
    
    root = "/work/data/input/google-smartphone-decimeter-challenge/"
    
    if (os.path.exists(f"{root}/train/{collection}/{phone}/{phone}_derived.csv")):
        file_path = f"{root}/train/{collection}/{phone}/{phone}_derived.csv"
    elif (os.path.exists(f"{root}/test/{collection}/{phone}/{phone}_derived.csv")):
        file_path = f"{root}/test/{collection}/{phone}/{phone}_derived.csv"
    else:
        raise Exception(f"not found: collection = {collection}, phone = {phone}");
                
    df_train = pd.read_csv(file_path)
    
    
    light_speed = 299_792_458                
    
    df_train['correctedPrM'] = df_train["rawPrM"] +  df_train["satClkBiasM"]  - df_train["isrbM"] - df_train["ionoDelayM"] - df_train["tropoDelayM"]
            
    
    ### Rotating the Satellite Reference Frame
    df_train['transmissionTimeSeconds'] = df_train['correctedPrM'] / light_speed    
    omega_e = 7.2921151467e-5
    df_train["xSatPosMRotated"] = np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    df_train["ySatPosMRotated"] = - np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] + np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    df_train['zSatPosMRotated'] = df_train['zSatPosM']
    
    #display(df_train)
    df_train['uncertaintyWeight'] = 1 / df_train['rawPrUncM']
        
                
    test_list = [];        
    exe_list = [[key,each_df]  for key,each_df in df_train.groupby("millisSinceGpsEpoch")];        
    num = multi(predict_by_satelite_per_time,exe_list)
    
                
    return pd.DataFrame(np.array(num),columns=["millisSinceGpsEpoch","x_pred","y_pred","z_pred"]);

    
collection,phone = df_baseline["phone"].unique()[0].split("_")
#c = df_baseline[df_baseline["phone"] == collection+"_"+phone];
 
collection = "2020-05-29-US-MTV-1"
phone = "Pixel4"
gnss = predict_by_gnss_data(c,"2020-05-29-US-MTV-1","Pixel4");

CPU times: user 3 s, sys: 2.62 s, total: 5.62 s
Wall time: 3min 46s


In [109]:
gnss["millisSinceGpsEpoch"] = gnss["millisSinceGpsEpoch"].astype("int64")

In [110]:
gnss.shape

(1912, 4)

In [111]:
#val = np.array(gnss)
#print(val)
#b_path = f"{root}/train/{collection}/{phone}/{phone}_derived.csv"
#b = pd.read_csv(b_path)
#display(b["millisSinceGpsEpoch"])
test = pd.read_csv(f"{root}/train/{collection}/{phone}/ground_truth.csv")
display(test["millisSinceGpsEpoch"])
1274827487
#for s in val[:,0]:
#    print(s)

0       1274827486438
1       1274827487438
2       1274827488438
3       1274827489438
4       1274827490438
            ...      
1908    1274829394439
1909    1274829395439
1910    1274829396439
1911    1274829397439
1912    1274829398439
Name: millisSinceGpsEpoch, Length: 1913, dtype: int64

1274827487

In [112]:
merged = pd.merge_asof(gnss, test, on='millisSinceGpsEpoch', direction='nearest', tolerance=1000)

s = merged.dropna()
s.shape

s.head(3)

from external_lib.evaluation_function import calc_haversine,evaluate_function


s["dist"] = calc_haversine(s["x_pred"],s["y_pred"],s["latDeg"],s["lngDeg"])
print(s["dist"].mean())


s[["latDeg_gt","lngDeg_gt"]] = s[["latDeg","lngDeg"]]
s[["latDeg","lngDeg"]] = s[["x_pred","y_pred"]]

29.916201303307812


In [ ]:
#29.916201303307812

In [113]:
from external_lib.visualize import visualize_trafic
visualize_trafic(s)

In [81]:
visualize_trafic(extracted_train)

In [76]:
from lib.io import load_pickle_data
df_train_baseline = load_pickle_data("/work/data/input/selfmade_dataset/baseline_with_derived_data_v5/train.pkl")
extracted_train = df_train_baseline[df_train_baseline["phone"] == "2020-05-29-US-MTV-1_Pixel4"]

extracted_train["dist"] = calc_haversine(extracted_train["latDeg_gt"],extracted_train["lngDeg_gt"],extracted_train["latDeg"],extracted_train["lngDeg"])
print(extracted_train["dist"].mean())

# 1274827486438 1274827487438

3.8450361884593347


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [99]:

from tqdm.notebook import tqdm


def prog(calc_,list_):
    p = mp.Pool(mp.cpu_count()-2);
    
    s = p.map(calc_,list_);
    p.close();
    return s;

def calc(s):
    return s**2;


b = prog(calc,[i for i in range(10)])
print(b)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
